NLP project

In [16]:
import sys  
sys.path.insert(1, './transformer')

In [64]:
from dataset import *
from utils import *

In [58]:
input_path = './data/gec-only/train.src.tok'
target_path = './data/gec-only/train.tgt.tok'

dataset = construct_dataset(input_path, target_path, 856*2, 16)

In [51]:
len(dataset)

2047

In [53]:
for (batch, (input_ids, target_ids)) in enumerate(dataset):
    print(batch)
    print(input_ids)
    print(target_ids)
    break

0
tf.Tensor(
[[    0 10431 47489 ...     1     1     1]
 [    0 47447    13 ...     1     1     1]
 [    0 25417 17772 ...     1     1     1]
 ...
 [    0 36765 25482 ...     1     1     1]
 [    0 36709 34251 ...     1     1     1]
 [    0 25417 10965 ...     1     1     1]], shape=(16, 1711), dtype=int32)
tf.Tensor(
[[10431 47489 50118 ...     1     1     1]
 [47447    13  1470 ...     1     1     1]
 [25417 17772 36709 ...     1     1     1]
 ...
 [36765 25482 36709 ...     1     1     1]
 [36709 34251 22063 ...     1     1     1]
 [25417 10965 41171 ...     1     1     1]], shape=(16, 1711), dtype=int32)


In [63]:
import tensorflow as tf
from transformer.distilroberta_encoder import DistilRoBERTaEncoder
from transformer.decoder import Decoder


class Transformer(tf.keras.Model):

    def __init__(self, num_layers=None, d_model=None, num_heads=None, dff=None,
                 target_vocab_size=None, pe_target=None, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = DistilRoBERTaEncoder(d_model=d_model)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                               target_vocab_size, pe_target, rate)
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)


    def call(self, input_ids, target_ids, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(input_ids, training)

        dec_output, attention_weights = self.decoder(
            target_ids, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


In [66]:
# Initialize the Transformer model
num_layers = 6
d_model = 512
num_heads = 8
dff = 2048
target_vocab_size = len(dataset)
pe_target = 1711
rate = 0.01
target_vocab_size = len(target_ids)
transformer_model = Transformer(num_layers, d_model, num_heads, dff,
                                target_vocab_size, pe_target, rate)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [67]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [68]:
# Define loss function, optimizer, and other necessary components (as shown in the previous response)

# Training step function
@tf.function
def train_step(input_ids, target_ids):
    tar_inp = target_ids[:, :-1]
    tar_real = target_ids[:, 1:]

    # Create masks
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(input_ids, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer_model(input_ids, tar_inp, True, enc_padding_mask, combined_mask, dec_padding_mask)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer_model.trainable_variables))

    return loss

In [69]:
# Training loop
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0

    for (batch, (input_ids, target_ids)) in enumerate(dataset):  # Assuming dataset is prepared
        batch_loss = train_step(input_ids, target_ids)
        total_loss += batch_loss

        if batch % 100 == 0:
            print(f'Epoch {epoch + 1} Batch {batch} Loss {batch_loss.numpy():.4f}')

    print(f'Epoch {epoch + 1} Loss {total_loss / (batch + 1):.4f}')

OperatorNotAllowedInGraphError: in user code:

    File "C:\Users\a.hak\AppData\Local\Temp\ipykernel_57348\3181115009.py", line 13, in train_step  *
        predictions, _ = transformer_model(input_ids, tar_inp, True, enc_padding_mask, combined_mask, dec_padding_mask)
    File "C:\Users\a.hak\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    OperatorNotAllowedInGraphError: Exception encountered when calling layer 'transformer_1' (type Transformer).
    
    in user code:
    
        File "C:\Users\a.hak\JupiterNotebooks\nlp\./transformer\Transformer.py", line 21, in call  *
            dec_output, attention_weights = self.decoder(
        File "C:\Users\a.hak\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
            raise e.with_traceback(filtered_tb) from None
    
        OperatorN